# Funciones Auxiliares

In [ ]:
import numpy as np
import math
import cv2
import os


def PSNR(orig, reconstr):
    mse = np.mean((orig.astype(float) - reconstr.astype(float)) ** 2)
    if mse != 0:
        max_pixel = 255.0
        return 20 * math.log10(max_pixel / math.sqrt(mse))
    else:
        return 1

def gen_dataset(filenames, scale):
    # The model trains on 17x17 patches
    crop_size_lr = 17
    crop_size_hr = 17 * scale

    for p in filenames:
        image_decoded = cv2.imread(p.decode(), 3).astype(np.float32) / 255.0
        imgYCC = cv2.cvtColor(image_decoded, cv2.COLOR_BGR2YCrCb)
        cropped = imgYCC[0:(imgYCC.shape[0] - (imgYCC.shape[0] % scale)),
                  0:(imgYCC.shape[1] - (imgYCC.shape[1] % scale)), :]
        lr = cv2.resize(cropped, (int(cropped.shape[1] / scale), int(cropped.shape[0] / scale)),
                        interpolation=cv2.INTER_CUBIC)

        hr_y = imgYCC[:, :, 0]
        lr_y = lr[:, :, 0]

        numx = int(lr.shape[0] / crop_size_lr)
        numy = int(lr.shape[1] / crop_size_lr)
        for i in range(0, numx):
            startx = i * crop_size_lr
            endx = (i * crop_size_lr) + crop_size_lr
            startx_hr = i * crop_size_hr
            endx_hr = (i * crop_size_hr) + crop_size_hr
            for j in range(0, numy):
                starty = j * crop_size_lr
                endy = (j * crop_size_lr) + crop_size_lr
                starty_hr = j * crop_size_hr
                endy_hr = (j * crop_size_hr) + crop_size_hr

                crop_lr = lr_y[startx:endx, starty:endy]
                crop_hr = hr_y[startx_hr:endx_hr, starty_hr:endy_hr]

                hr = crop_hr.reshape((crop_size_hr, crop_size_hr, 1))
                lr = crop_lr.reshape((crop_size_lr, crop_size_lr, 1))
                yield lr, hr

# Parámetros

In [ ]:
scale = 2
epoch = 3
testimg = "/content/drive/MyDrive/TheLastofUs/Test/orig1.png"
lr = 0.0001
main_ckpt_dir = "/content/drive/MyDrive/checkpoints"
if not os.path.exists(main_ckpt_dir):
  os.makedirs(main_ckpt_dir)

traindir="/content/drive/MyDrive/TheLastofUs/HD3"

ARGS = dict()
ARGS["SCALE"] = int(scale)
ARGS["CKPT_dir"] = main_ckpt_dir + "/checkpoint" + "_sc" + str(scale)
ARGS["CKPT"] = ARGS["CKPT_dir"] + "/ESPCN_ckpt_sc" + str(scale)
ARGS["TRAINDIR"] = traindir
ARGS["EPOCH_NUM"] = int(epoch)
ARGS["TESTIMG"] = testimg
ARGS["LRATE"] = float(lr)



config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

# Entrenamiento

In [ ]:
import tensorflow as tf
import pathlib
import cv2
import numpy as np
import os
import math

from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

tf.compat.v1.disable_eager_execution()
def training(ARGS):
    """
    Start training the ESPCN model.
    """

    print("\nStarting training...\n")

    SCALE = ARGS["SCALE"]
    BATCH = 16
    EPOCHS = ARGS["EPOCH_NUM"]
    DATA = pathlib.Path(ARGS["TRAINDIR"])
    LRATE = ARGS["LRATE"]

    all_image_paths = list(DATA.glob('*'))
    all_image_paths = [str(path) for path in all_image_paths]

    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True

    ds = tf.data.Dataset.from_generator(
        gen_dataset, (tf.float32, tf.float32), (tf.TensorShape([None, None, 1]), tf.TensorShape([None, None, 1])),
        args=[all_image_paths, SCALE])
    train_dataset = ds.batch(BATCH)
    train_dataset = train_dataset.shuffle(10000)
    #iter = train_dataset.make_initializable_iterator()
    iter = tf.compat.v1.data.make_initializable_iterator(train_dataset)
    LR, HR = iter.get_next()
    EM = ESPCN(input=LR, scale=SCALE, learning_rate=LRATE)
    model = EM.ESPCN_model()
    loss, train_op, psnr = EM.ESPCN_trainable_model(model, HR)

    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        train_writer = tf.summary.FileWriter('./logs/train', sess.graph)

        saver = EM.saver
        if not os.path.exists(ARGS["CKPT_dir"]):
            os.makedirs(ARGS["CKPT_dir"])
        else:
            if os.path.isfile(ARGS["CKPT"] + ".meta"):
                saver.restore(sess, tf.train.latest_checkpoint(ARGS["CKPT_dir"]))
                print("Loaded checkpoint.")
            else:
                print("Previous checkpoint does not exists.")

        # training with tf.data method
        saver = tf.train.Saver()
        for e in range(EPOCHS):
            sess.run(iter.initializer)
            count = 0
            train_loss, train_psnr = 0.0, 0.0
            while True:
                try:
                    count = count + 1
                    l, t, ps = sess.run([loss, train_op, psnr])
                    train_loss += l
                    train_psnr += (np.mean(np.asarray(ps)))

                    if count % 1000 == 0:
                        print("Data num:", '%d' % (count), "Epoch no:", '%04d' % (e + 1), "loss:", "{:.9f}".format(l),
                              "epoch loss:", "{:.9f}".format(train_loss / (count)),
                              "epoch psnr:", "{:.9f}".format(train_psnr / (count)))
                        saver.save(sess, ARGS["CKPT"])

                except tf.errors.OutOfRangeError:
                    break
            print("END OF EPOCH - Epoch no:", '%04d' % (e + 1), "loss:", "{:.9f}".format(float(l)),
                  "epoch loss:", "{:.9f}".format(train_loss / (count)),
                  "epoch psnr:", "{:.9f}".format(train_psnr / (count)))
            saver.save(sess, ARGS["CKPT"])

        train_writer.close()



# Estructura ESPCN 

In [ ]:
import tensorflow as tf
import os


class ESPCN:

    def __init__(self, input, scale, learning_rate):
        self.LR_input = input
        self.scale = scale
        self.learning_rate = learning_rate
        self.saver = ""

    def ESPCN_model(self):
        """
        Implementation of ESPCN: https://arxiv.org/abs/1609.05158

        Returns
        ----------
        Model
        """

        scale = self.scale
        channels = 1
        bias_initializer = tf.constant_initializer(value=0.1)
        initializer = tf.contrib.layers.xavier_initializer_conv2d()

        filters = [
            tf.Variable(initializer(shape=(5, 5, channels, 64)), name="f1"),  # (f1,n1) = (5,64)
            tf.Variable(initializer(shape=(3, 3, 64, 32)), name="f2"),  # (f2,n2) = (3,32)
            tf.Variable(initializer(shape=(3, 3, 32, channels * (scale * scale))), name="f3")  # (f3) = (3)
        ]

        bias = [
            tf.get_variable(shape=[64], initializer=bias_initializer, name="b1"),
            tf.get_variable(shape=[32], initializer=bias_initializer, name="b2"),
            tf.get_variable(shape=[channels * (scale * scale)], initializer=bias_initializer, name="b3")  # HxWxr^2
        ]

        l1 = tf.nn.conv2d(self.LR_input, filters[0], [1, 1, 1, 1], padding='SAME', name="conv1")
        l1 = l1 + bias[0]
        l1 = tf.nn.relu(l1)

        l2 = tf.nn.conv2d(l1, filters[1], [1, 1, 1, 1], padding='SAME', name="conv2")
        l2 = l2 + bias[1]
        l2 = tf.nn.relu(l2)

        l3 = tf.nn.conv2d(l2, filters[2], [1, 1, 1, 1], padding='SAME', name="conv3")
        l3 = l3 + bias[2]

        # Depth_to_space is equivalent to the pixel shuffle layer.
        out = tf.nn.depth_to_space(l3, scale, data_format='NHWC')

        out = tf.nn.tanh(out, name="NHWC_output")

        out_nchw = tf.transpose(out, [0, 3, 1, 2], name="NCHW_output")
        # out = tf.nn.relu(out, name="NHWC_output")

        self.saver = tf.train.Saver()

        return out

    def ESPCN_trainable_model(self, HR_out, HR_orig):
        psnr = tf.image.psnr(HR_out, HR_orig, max_val=1.0)

        loss = tf.losses.mean_squared_error(HR_orig, HR_out)

        train_op = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(loss)

        return loss, train_op, psnr
